In [ ]:
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (6, 6)

import math
import cmath          # math functions for complex numbers
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm

import scipy
import scipy.stats
import pandas as pd

from astropy.io import fits
import os.path

import ipywidgets
from ipywidgets import interact

import sys
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline")
import datapipe
import datapipe.denoising.wavelets_mrfilter as mrfilter
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils")
import common_functions as common

## Search for good images

In [ ]:
#CSV_FILE_PATH = "../xps/best.csv"
CSV_FILE_PATH = "../xps/2017_05_12/2017_05_12.csv"
#CSV_FILE_PATH = "../xps/2017_05_15_cropped_fits_files/2017_05_15_cropped_fits_files.csv"

#WAVELET_LABEL = "WT-K-k-C1-m3-n4-s3"
WAVELET_LABEL = "WT-K-k-C1-m3-n4-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10"

PART = 0       # 0 for gamma, 1 for protons

full_df = pd.read_csv(CSV_FILE_PATH)

full_df.columns

In [ ]:
full_df

In [ ]:
ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

## Set the selection

In [ ]:
#selection = df[df.peSum_tc > 50][df.peSum_tc < 200][df.nPix_tc > 10][df.delta_tc > 5.][df.delta_wt < 1.].loc[:,['Id','Tel','Evt','delta_tc','delta_wt']]
#selection = df[df.nPix > 1000].loc[:,['Id','Tel','Evt','delta_tc','delta_wt']]
selection = df[df.peSum > 50][df.peSum < 55].loc[:,['Id','Tel','Evt','delta_tc','delta_wt','fits','cam']]
print("Search:", len(selection))
print("Search:", selection)

In [ ]:
file_path_list = []
for fits, cam_id in zip(selection.fits.values, selection.cam.values):
    if PART == 0:
        if cam_id == "ASTRI":
            dir_path = "/Users/jdecock/data/astri_mini_array/fits/gamma"
        elif cam_id == "ASTRI_CROPPED":
            dir_path = "/Users/jdecock/data/astri_mini_array/fits_cropped/gamma"
    elif PART == 1:
        if cam_id == "ASTRI":
            dir_path = "/Users/jdecock/data/astri_mini_array/fits/proton"
        elif cam_id == "ASTRI_CROPPED":
            dir_path = "/Users/jdecock/data/astri_mini_array/fits_cropped/proton"

    file_name = fits

    path = os.path.join(dir_path, file_name)
    file_path_list.append(path)

print(file_path_list)

## GUI

In [ ]:
# mr_filter options

option_list = [
    "-K -C1 -m3  -s2,2,3,3 -n4",
    "-K -C1 -m3  -s4 -n4 -t24 -f3",           # Suggested by Jean-Luc (TODO: try to adapt -s)
#    "-K -C1 -m3  -s5 -n4 -t28 -f3 -i10 -e0",  # Suggested by Jean-Luc (TODO: try to adapt -s)
    "-K -C1 -m3  -s3 -n4",      # Hard K-Sigma Thresholding, Poisson + Gaussian
    "-K -C1 -m3  -s3 -n4 -f3",
#    "-K -C1 -m3  -s2,2,3,3 -n4 -f3",
#    "-K -C1 -m3  -s3 -n4 -f2",  # Soft K-Sigma Thresholding, Poisson + Gaussian (deconseille par JL)
#    "-K -C2 -m1      -n4 -f2",  # False Discovery Rate
#    "-K     -m1      -n4 -f6",  # Wiener Filtering (min MSE between random process and desired process)
#    "-K -C1 -m10 -s3 -n4",      # Poisson with few events
#    "-K -C1 -m2  -s3 -n4",      # Poisson
    ]

In [ ]:
@interact(kill_pix=True, hist=False, log_scale=True, ellipses=True, lateral_hst=["Wavelet", "Tailcut", None], ref_angle=False, file_path=file_path_list, mrfilter=option_list)
def gui(kill_pix, hist, log_scale, ellipses, lateral_hst, ref_angle, file_path, mrfilter):
    fits_images_dict, fits_metadata_dict = datapipe.io.images.load_benchmark_images(file_path)
    input_img = fits_images_dict["input_image"]
    reference_img = fits_images_dict["reference_image"]
    pixels_position = fits_images_dict["pixels_position"]

    fig = plt.figure(figsize=(14, 14))

    common.plot_gui(fig,
                    input_img,
                    reference_img,
                    pixels_position,
                    fits_metadata_dict,
                    wavelets_cmd=mrfilter,
                    kill_isolated_pixels=kill_pix,
                    plot_histogram=hist,
                    plot_log_scale=log_scale,
                    plot_ellipse_shower=ellipses,
                    _plot_perpendicular_hit_distribution=lateral_hst,  # None, "Tailcut", "Wavelet"
                    use_ref_angle_for_perpendicular_hit_distribution=ref_angle,
                    notebook=True
                   )